# Ingest Real-Time Stock Data to Iguazio NoSQL and Time-series DB
the following example function ingest real-time stock information from an internet service (Yahoo finance api) into iguazio platform.<br>
everytime the data is updated it updates a NoSQL table with the recent metadata and updates the time-series DB with the new metrics (price and volume)

The same code can run inside a nuclio (serverless) function and be automatically triggered on a predefined schedule (cron) or through HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations (such as the cron schedule), and to deploy functions automatically onto a cluster.

In [1]:
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 
import os

## Environment

copy the local credentials to the nuclio function config (-c option doesn't initialize locally)

In [18]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [19]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [20]:
%%nuclio cmd -c
pip install lxml
pip install yfinance
pip install requests
pip install v3io_frames

In [54]:
!pip install pandas==1.2.3

     |████████████████████████████████| 9.9 MB 5.2 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3


## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [6]:
# nuclio: start-code

In [271]:
import yfinance as yf
import os
import pandas as pd
import v3io_frames as v3f
import ast
import mlrun.feature_store as fs
import mlrun
import datetime
from mlrun.datastore.targets import ParquetTarget

In [170]:
start = datetime.datetime.now()-datetime.timedelta(5)
end = datetime.datetime.now()

In [172]:
yf.Ticker("GOOGL").history(start=start, end=end,interval = "1m")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-06-10 09:30:00-04:00,2406.919922,2410.000000,2403.780029,2410.000000,24985,0,0
2021-06-10 09:31:00-04:00,2410.750000,2416.209961,2408.465088,2416.010010,8467,0,0
2021-06-10 09:32:00-04:00,2414.790039,2416.729980,2413.947998,2415.665039,5223,0,0
2021-06-10 09:33:00-04:00,2416.520020,2418.860107,2416.399902,2418.669922,6340,0,0
2021-06-10 09:34:00-04:00,2418.574951,2420.320068,2418.574951,2419.379883,5728,0,0
...,...,...,...,...,...,...,...
2021-06-14 15:55:00-04:00,2447.617920,2447.617920,2446.340088,2446.340088,10902,0,0
2021-06-14 15:56:00-04:00,2445.870117,2446.790039,2445.449951,2445.899902,10903,0,0
2021-06-14 15:57:00-04:00,2445.770020,2446.949951,2445.669922,2446.949951,9279,0,0


In [272]:
def update_tickers(context, start, end, interval):
    all_records=[]
    stocks_df = pd.DataFrame()
    for sym in context.stock_syms:
        hist = yf.Ticker(sym).history(start=start, end=end,interval=interval).drop(["Dividends","Stock Splits"],axis=1)
        time = hist.index[len(hist) - 1]
        record = hist.loc[time]
        last = context.last_trade_times.get(sym)
        if last:
            hist = hist[hist.index>last]
        context.logger.info(f'Received {sym} data from yfinance, including {len(hist)} candles ending at {last}')
        # update the stocks table and TSDB metrics in case of new data
        if not last or time > last:
            fs_data = hist.copy().reset_index()
            fs_data["symbol"]=sym
            fs_data.set_index("symbol",inplace=True)
            # ingesting new data to a featureset
            context.logger.info(f"Writing new dataframe with shape {fs_data.shape} to feature store")
            fs.ingest(context.stock_info_feature_set, fs_data, infer_options=fs.InferOptions.default())
            
            # update NoSQL table with stock data
            stock = {'symbol': sym, 'price': record['Close'], 'volume': record['Volume'], 'last_updated': time}
            #all_records.append(stock)
            expr = context.expr_template.format(**stock)
            context.logger.debug_with('update expression', symbol=sym, expr=expr)
            context.v3c.execute('kv', context.stocks_kv_table, 'update', args={'key': sym, 'expression': expr})
         
            context.logger.info(f'Updated records from {last} to {time}')
            # update time-series DB with price and volume metrics (use pandas dataframe with a single row, indexed by date)
            context.last_trade_times[sym] = time
            hist['symbol'] = sym
            hist = hist.reset_index()
            hist = hist.set_index(['Datetime', 'symbol'])
            hist = hist.loc[:, ['Close', 'Volume']]
            hist = hist.rename(columns={'Close': 'price', 'Volume': 'volume'})
            stocks_df = stocks_df.append(hist)
            context.logger.info(f'Added records {hist.shape[0]} records for {sym} to history')
        else:
            context.logger.info(f'No update was made, current TS: {last} vs. new data {time}')
    
    # inferring KV to create a scheme
#     context.v3c.execute("kv",table = context.stocks_kv_table, command = "infer")
    
    # write price and volume metrics to the Time-Series DB, add exchange label
    if stocks_df.shape[0]>0:
        stocks_df = stocks_df.sort_index(level=0)
        context.logger.debug_with('writing data to TSDB', stocks=stocks_df)
        stocks_df.to_csv('history.csv')
        context.v3c.write(backend='tsdb', table=context.stocks_tsdb_table, dfs=stocks_df)


In [277]:
def init_context(context):
    context.logger.info("init stocks reader context")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project = context.PROJECT_NAME)
    # Setup V3IO Client
    setattr(context,"V3IO_FRAMESD", os.getenv("V3IO_FRAMESD",'framesd:8081'))
    client = v3f.Client(context.V3IO_FRAMESD, container=os.getenv('V3IO_CONTAINER', 'users'))
    setattr(context, 'v3c', client)
    
    # Create V3IO Tables and add reference to context
    setattr(context, 'stocks_kv_table', os.getenv('STOCKS_KV_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_kv'))
    setattr(context, 'stocks_tsdb_table', os.getenv('STOCKS_TSDB_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb'))
    context.v3c.create(backend='tsdb', table=context.stocks_tsdb_table, rate='1/m', if_exists=1)
    
    # Supply the feature set to ingest data to.
    stocks_info_set = fs.FeatureSet("stocks", entities=[fs.Entity("symbol")],timestamp_key="Datetime")
    stocks_info_set.set_targets(targest=[ParquetTarget(partitioned=True,time_partitioning_granularity=[None, "day"])])
    setattr(context,'stock_info_feature_set',stocks_info_set)
    
    # Adding aggregations
    context.stock_info_feature_set.add_aggregation("Closes","Close",["min","max"],["1h"])
    context.stock_info_feature_set.add_aggregation("Volumes","Volume",["min","max"],["1h"])
    
    # Initiazling featureset with dummy data that will be overtwritten later on
#     stock_dummy = pd.DataFrame({"symbol":['GOOGL','MSFT','AMZN','AAPL','INTC'],"price":[0,0,0,0,0],"volume":[0,0,0,0,0],"last_updated":[0,0,0,0,0]})
#     fs.ingest(context.stock_info_feature_set, stock_dummy, infer_options=fs.InferOptions.default())
    
    stocks = os.getenv('STOCK_LIST','GOOGL,MSFT,AMZN,AAPL,INTC')
    if stocks.startswith('['):
        stock_syms = ast.literal_eval(stocks)
    else: 
        stock_syms = stocks.split(',')
    setattr(context, 'stock_syms', stock_syms)
    

    # v3io update expression template 
    expr_template = os.getenv('EXPRESSION_TEMPLATE', "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'")
    setattr(context, 'expr_template', expr_template)

    last_trade_times = {}
    setattr(context, 'last_trade_times', last_trade_times)
    
    # Run first initial data preperation
    start = datetime.datetime.now()-datetime.timedelta(7)
    end = datetime.datetime.now()-datetime.timedelta(3)
    s_df = update_tickers(context, start=start, end=end,interval="1m")
    

In [278]:
def handler(context):
    start = datetime.datetime.now()-datetime.timedelta(5)
    end = datetime.datetime.now()
    stocks_df = update_tickers(context, start=start, end=end,interval="1m")
    return stocks_df

In [279]:
# nuclio: end-code

## Function invocation
### Local test
the following section simulates nuclio function invocation and will emit the function results

In [280]:
# create a test event and invoke the function locally 
s_df1=init_context(context)
s_df2=handler(context)

Python> 2021-06-15 12:02:34,761 [info] init stocks reader context


TypeError: __init__() got an unexpected keyword argument 'partitioned'

In [206]:
s_df1

In [207]:
s_df2

,Datetime,Open,High,Low,Close,Volume
symbol,,,,,,
GOOGL,2021-06-10 09:30:00-04:00,2406.919922,2410.000000,2403.780029,2410.000000,24985
GOOGL,2021-06-10 09:31:00-04:00,2410.750000,2416.209961,2408.465088,2416.010010,8467
GOOGL,2021-06-10 09:32:00-04:00,2414.790039,2416.729980,2413.947998,2415.665039,5223
GOOGL,2021-06-10 09:33:00-04:00,2416.520020,2418.860107,2416.399902,2418.669922,6340
GOOGL,2021-06-10 09:34:00-04:00,2418.574951,2420.320068,2418.574951,2419.379883,5728
...,...,...,...,...,...,...
GOOGL,2021-06-14 15:55:00-04:00,2447.617920,2447.617920,2446.340088,2446.340088,10902
GOOGL,2021-06-14 15:56:00-04:00,2445.870117,2446.790039,2445.449951,2445.899902,10903
GOOGL,2021-06-14 15:57:00-04:00,2445.770020,2446.949951,2445.669922,2446.949951,9279


## Deploy to cluster

In [7]:
from mlrun import code_to_function

# Export bare function
fn = code_to_function('read-stocks',
                      handler='handler')
fn.export('01-read-stocks.yaml')

# Set parameters for current deployment
fn.add_trigger('cron', nuclio.triggers.CronTrigger('300s'))
fn.set_envs({'STOCK_LIST': ['GOOG', 'MSFT', 'AMZN', 'AAPL', 'INTC'],
             'V3IO_CONTAINER': 'users' ,
             'STOCKS_TSDB_TABLE': os.getenv('V3IO_USERNAME')  + 'stocks/stocks_tsdb',
             'STOCKS_KV_TABLE': os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
             'EXPRESSION_TEMPLATE': "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'",
             'PROJECT_NAME' : "stocks-" + os.getenv('V3IO_USERNAME')})
fn.spec.max_replicas = 1

> 2021-03-25 11:00:46,655 [info] function spec saved to path: 01-read-stocks.yaml


In [8]:
addr = fn.deploy(project='stocks-dani')

> 2021-03-25 11:00:56,616 [info] Starting remote function deploy
2021-03-25 11:00:56  (info) Deploying function
2021-03-25 11:00:56  (info) Building
2021-03-25 11:00:57  (info) Staging files and preparing base images
2021-03-25 11:00:57  (info) Building processor image
2021-03-25 11:00:58  (info) Build complete
2021-03-25 11:01:21  (info) Function deploy complete
> 2021-03-25 11:01:21,724 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:31840


In [10]:
!curl {addr}

done